# Step 1. 데이터 다운로드
## ~/aiffel/lyricist/data/lyrics에 데이터를 다운 받았다.

# Step 2. 데이터 읽어오기

In [1]:
import glob
import os, re
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


# Step 3. 데이터 정제

In [2]:
def preprocess_sentence(sentence):
    #sentence = re.sub(r'\[[^)]*\]', "", sentence) # [] 사이의 문자 제거
    sentence = sentence.lower().strip()  # 소문자, 양쪽공백 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence) # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip() # 양쪽 공백 제거
    sentence = '<start> ' + sentence + ' <end>' 
    return sentence

In [3]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == "]": continue
    if len(preprocess_sentence(sentence).split()) > 15: continue
        
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:20]

['<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> look like you hidin your problems <end>',
 '<start> really you never was solid <end>',
 '<start> no , you can t son me <end>',
 '<start> you won t never get to run me <end>',
 '<start> just when shit look out of reach <end>',
 '<start> i reach back like one , three <end>',
 '<start> that s when they smile in my face <end>',
 '<start> whole time they wanna take my place <end>',
 '<start> whole time they wanna take my place <end>',
 '<start> whole time they wanna take my place <end>',
 '<sta

In [4]:
def tokenize(corpus):
    # 12000 단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000 단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    4   95 ...   10   12    3]
 [   2   38  133 ...    0    0    0]
 [   2    4   36 ...    0    0    0]
 ...
 [   2  204    3 ...    0    0    0]
 [   2  414    9 ...    0    0    0]
 [   2    9 1552 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f541b07ddd0>


In [5]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2   4  95 105  58  31 168   5  11 133  23  29  10  12]
[  4  95 105  58  31 168   5  11 133  23  29  10  12   3]


# Step 4. 평가 데이터셋 분리

In [6]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=20)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124076, 14)
Target Train: (124076, 14)


# Step 5. 인공지능 만들기

In [7]:
# 모델 만들기
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 3000
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [8]:
# 모델 학습
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none')


model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=4, validation_data=(enc_val, dec_val))

Epoch 1/4
3878/3878 [==============================] - 1754s 438ms/step - loss: 3.1691 - val_loss: 2.6030
Epoch 2/4
3878/3878 [==============================] - 1698s 438ms/step - loss: 2.3559 - val_loss: 2.3333
Epoch 3/4
3878/3878 [==============================] - 1695s 437ms/step - loss: 1.8124 - val_loss: 2.1987
Epoch 4/4
3878/3878 [==============================] - 1692s 436ms/step - loss: 1.4279 - val_loss: 2.1796


In [9]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [10]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love the way you lie <end> '

# 회고
## - 데이터 정제하는 과정이 어려웠다. 학습데이터 갯수가 124960보다 작아야 하는데, 처음에는 124981이 나와서 제거하지 못한 데이터가 무엇인지 계속 찾았다.  


## - 원본 데이터를 보는데, []사이에 Hook, Verse, Corus 등 가사가 아닌 단어들이 적혀 있어서 괄호 안에 있는 단어를 제거해야겠다고 생각했다. 처음에는 sentence = re.sub(r'\[[^)]*\]', "", sentence)를 정제 과정에서 추가해서 괄호 사이에 있는 단어를 제거했다. 하지만 데이터 갯수가 125037로 오히려 더 많이 나왔다. 아직도 더 많이 나오는 이유에 대해서는 잘 모르겠다.

## - 두번째로 생각한 것이 보통 문장의 마지막에 [Verse], [Corus] 등이 추가 되어 있으므로, 마지막 문자가 만약에 "]"이라면 아예 그 문장을 제거하는 방식으로 했다. 이 방식으로 하니까 데이터 갯수가 124076으로 124960보다 더 작게 나왔다. 

## - 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기를 권한다고 했는데, 처음에는 토큰의 개수를 세는 방법을 몰라서 많이 헤메다가 검색을 통해서 알아냈다. len(preprocess_sentence(sentence).split())이 토큰화 했을 때의 토큰 개수로 .split()를 사용한다.

## - embedding_size = 512, hidden_size = 1024, epoch = 10으로 했을 때, 2.4295의 로스가 발생했다. 결과적으로는 2.2보다 높은 로스를 보였지만, epoch를 반복할 때마다 로스가 점점 줄어들다가 epoch = 4 일 때가 제일 낮고 다시 점점 높아지는 형상을 띄었다. 따라서 embedding_size = 512, hidden_size = 3000, epoch = 4로 바꿨고, 결과적으로 loss가 2.1796가 나왔다.
